### **特徴量 ver2**
店舗の情報, 商品の情報, イベント情報, 日時情報, 価格情報, One-HotEncoding

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
from myUtils import reduce_mem_usage,encode_categorical,encode_onehot

import gc
mpl.rcParams['agg.path.chunksize'] = 100000

#### **ベースのデータ**
ここに情報をくっつけていく

In [2]:
data = pd.read_pickle(os.path.join("mydata","sales_train_val.pickle")) #preprocessing.ipynbで作成したデータ
data.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d,item_cnt,data_part
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train


#### **カレンダー情報**

In [3]:
calendar = pd.read_csv(os.path.join("rawdata","calendar.csv"))
print("**** calendar ****")
display(calendar.head(5))

**** calendar ****


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


#### **価格の情報を結合**
- 各商品, 各店舗ごとに, 週ごとの価格が入ってる. 
- 週番号11101～11324の値がないので, 商品, 店舗ごとに価格の最大値, 最小値, 平均, 中央値を計算する

In [4]:
sell_prices = pd.read_csv(os.path.join("rawdata","sell_prices.csv"))
sell_prices = sell_prices.groupby(by=["item_id","store_id"]).agg({"sell_price":["median","mean","max","min"]})
sell_prices = sell_prices.reset_index()
sell_prices.columns = ["item_id","store_id","price-median","price-mean","price-max","price-min"]
print("**** sell_prices ****")
display(sell_prices)

**** sell_prices ****


,item_id,store_id,price-median,price-mean,price-max,price-min
0,FOODS_1_001,CA_1,2.24,2.169362,2.24,2.00
1,FOODS_1_001,CA_2,2.24,2.169362,2.24,2.00
2,FOODS_1_001,CA_3,2.24,2.158262,2.24,1.75
3,FOODS_1_001,CA_4,2.24,2.169362,2.24,2.00
4,FOODS_1_001,TX_1,2.24,2.157624,2.24,0.99
...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516,TX_2,5.94,5.922482,5.94,3.00
30486,HOUSEHOLD_2_516,TX_3,5.94,5.922447,5.94,0.99
30487,HOUSEHOLD_2_516,WI_1,5.94,5.940000,5.94,5.94
30488,HOUSEHOLD_2_516,WI_2,5.94,5.940000,5.94,5.94


#### **カレンダー, 価格情報を付与する**

In [5]:
data = pd.merge(data,calendar,on=["d"],how="left") # カレンダー情報
data = pd.merge(data,sell_prices,on=["item_id","store_id"],how="left") # 価格情報

print("**** data(joined price-info) ****")
data = reduce_mem_usage(data)
display(data.head(5))

**** data(joined price-info) ****
Mem. usage decreased to 7958.26 Mb (33.8% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,item_cnt,data_part,date,...,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,price-median,price-mean,price-max,price-min
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train,2011-01-29,...,NaN,NaN,NaN,0,0,0,8.257812,8.289062,9.578125,8.257812
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train,2011-01-29,...,NaN,NaN,NaN,0,0,0,3.970703,3.970703,3.970703,3.970703
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train,2011-01-29,...,NaN,NaN,NaN,0,0,0,2.970703,2.970703,2.970703,2.970703
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train,2011-01-29,...,NaN,NaN,NaN,0,0,0,4.640625,4.527344,4.640625,4.339844
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,train,2011-01-29,...,NaN,NaN,NaN,0,0,0,2.980469,2.941406,3.080078,2.480469


#### **各カラム, NaNを埋める**

In [6]:
# 各特徴量 NaNの割合
print("**** ratio of NaN-values ****")
display(data.isna().sum()/len(data)*100)

**** ratio of NaN-values ****


id               0.000000
item_id          0.000000
dept_id          0.000000
cat_id           0.000000
store_id         0.000000
state_id         0.000000
d                0.000000
item_cnt         2.844083
data_part        0.000000
date             0.000000
wm_yr_wk         0.000000
weekday          0.000000
wday             0.000000
month            0.000000
year             0.000000
event_name_1    91.772473
event_type_1    91.772473
event_name_2    99.746064
event_type_2    99.746064
snap_CA          0.000000
snap_TX          0.000000
snap_WI          0.000000
price-median     0.000000
price-mean       0.000000
price-max        0.000000
price-min        0.000000
dtype: float64

In [7]:
# year,month,wday
data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["wday"] = data["date"].dt.weekday

#### **One-Hot Encoding**

In [9]:
cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
data = encode_onehot(data, cols=cols)
data = reduce_mem_usage(data)

display(data.head(5))

Mem. usage decreased to 5315.90 Mb (17.7% reduction)


,id,d,item_cnt,data_part,date,wm_yr_wk,weekday,wday,month,year,...,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster,event_type_2_Cultural,event_type_2_Religious
0,HOBBIES_1_001_CA_1_validation,d_1,0.0,train,2011-01-29,11101,Saturday,5,1,2011,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,d_1,0.0,train,2011-01-29,11101,Saturday,5,1,2011,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,d_1,0.0,train,2011-01-29,11101,Saturday,5,1,2011,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,d_1,0.0,train,2011-01-29,11101,Saturday,5,1,2011,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,d_1,0.0,train,2011-01-29,11101,Saturday,5,1,2011,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data.to_pickle(os.path.join("mydata","data_v2.pickle"))

In [21]:
list(data.columns)

['id',
 'd',
 'item_cnt',
 'data_part',
 'date',
 'wm_yr_wk',
 'weekday',
 'wday',
 'month',
 'year',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'price-median',
 'price-mean',
 'price-max',
 'price-min',
 'item_id_FOODS_1_001',
 'item_id_FOODS_1_002',
 'item_id_FOODS_1_003',
 'item_id_FOODS_1_004',
 'item_id_FOODS_1_005',
 'item_id_FOODS_1_006',
 'item_id_FOODS_1_008',
 'item_id_FOODS_1_009',
 'item_id_FOODS_1_010',
 'item_id_FOODS_1_011',
 'item_id_FOODS_1_012',
 'item_id_FOODS_1_013',
 'item_id_FOODS_1_014',
 'item_id_FOODS_1_015',
 'item_id_FOODS_1_016',
 'item_id_FOODS_1_017',
 'item_id_FOODS_1_018',
 'item_id_FOODS_1_019',
 'item_id_FOODS_1_020',
 'item_id_FOODS_1_021',
 'item_id_FOODS_1_022',
 'item_id_FOODS_1_023',
 'item_id_FOODS_1_024',
 'item_id_FOODS_1_025',
 'item_id_FOODS_1_026',
 'item_id_FOODS_1_027',
 'item_id_FOODS_1_028',
 'item_id_FOODS_1_029',
 'item_id_FOODS_1_030',
 'item_id_FOODS_1_031',
 'item_id_FOODS_1_032',
 'item_id_FOODS_1_033',
 'item_id_FOODS_1_034',
 'item_id_